In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fastai.text.all import *

https://medium.com/@alden_6876/getting-document-encodings-from-fastais-ulmfit-language-model-a3f9271f9ecd

In [5]:
path = untar_data(URLs.AG_NEWS)

In [19]:
df = pd.read_csv(path/'test.csv', header=None)

In [43]:
df.head()

,0,1,2,text,label
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Business
1,4,The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com),"SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.","The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.",Sci/Tech
2,4,Ky. Company Wins Grant to Study Peptides (AP),"AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.","Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.",Sci/Tech
3,4,Prediction Unit Helps Forecast Wildfires (AP),"AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.","Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.",Sci/Tech
4,4,Calif. Aims to Limit Farm-Related Smog (AP),"AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.","Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.",Sci/Tech


In [20]:
df_class = pd.read_csv(path/'classes.txt', header=None)

In [22]:
df_class.head()

,0
0,World
1,Sports
2,Business
3,Sci/Tech


In [21]:
df['text'] = df[1] + " " + df[2]

In [40]:
def f(a):
    return df_class.iloc[int(a-1)][0]

In [41]:
df['label'] = df[0].apply(f)

In [42]:
df.head()

,0,1,2,text,label
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Business
1,4,The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com),"SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.","The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.",Sci/Tech
2,4,Ky. Company Wins Grant to Study Peptides (AP),"AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.","Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.",Sci/Tech
3,4,Prediction Unit Helps Forecast Wildfires (AP),"AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.","Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.",Sci/Tech
4,4,Calif. Aims to Limit Farm-Related Smog (AP),"AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.","Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.",Sci/Tech


In [ ]:
learn = text_classification_learner(dls, AWD_LSTM, metrics=accuracy)

In [ ]:
learn.export('/Users/florian/Google Drive/ml/datasets/text/export_ag_news.pkl')

In [4]:
learn = load_learner('/Users/florian/Google Drive/ml/datasets/text/export_ag_news.pkl')

In [44]:
#from fastai.callback.tensorboard import *
import random
import tensorboard
from torch.utils.tensorboard import SummaryWriter

In [45]:
class TensorBoardBaseCallback(Callback):
    "Base class for tensorboard callbacks"
    def __init__(self):
        self.run_projector = False

    def after_pred(self):
        if self.run_projector: self.feat = _add_projector_features(self.learn, self.h, self.feat)

    def after_validate(self):
        if not self.run_projector: return
        self.run_projector = False
        self._remove()
        _write_projector_embedding(self.learn, self.writer, self.feat)

    def after_fit(self):
        if self.run: self.writer.close()

    def _setup_projector(self):
        self.run_projector = True
        self.h = hook_output(self.learn.model[1][1] if not self.layer else self.layer)
        self.feat = {}

    def _setup_writer(self):
        self.writer = SummaryWriter(log_dir=self.log_dir)

    def _remove(self):
        if getattr(self, 'h', None): self.h.remove()

    def __del__(self): self._remove()

# Cell
class TensorBoardCallback(TensorBoardBaseCallback):
    "Saves model topology, losses & metrics for tensorboard and tensorboard projector during training"
    def __init__(self, log_dir=None, trace_model=True, log_preds=True, n_preds=9, projector=False, layer=None):
        super().__init__()
        store_attr()

    def before_fit(self):
        self.run = not hasattr(self.learn, 'lr_finder') and not hasattr(self, "gather_preds") and rank_distrib()==0
        if not self.run: return
        self._setup_writer()
        if self.trace_model:
            if hasattr(self.learn, 'mixed_precision'):
                raise Exception("Can't trace model in mixed precision, pass `trace_model=False` or don't use FP16.")
            b = self.dls.one_batch()
            self.learn._split(b)
            self.writer.add_graph(self.model, *self.xb)

    def after_batch(self):
        self.writer.add_scalar('train_loss', self.smooth_loss, self.train_iter)
        for i,h in enumerate(self.opt.hypers):
            for k,v in h.items(): self.writer.add_scalar(f'{k}_{i}', v, self.train_iter)

    def after_epoch(self):
        for n,v in zip(self.recorder.metric_names[2:-1], self.recorder.log[2:-1]):
            self.writer.add_scalar(n, v, self.train_iter)
        if self.log_preds:
            b = self.dls.valid.one_batch()
            self.learn.one_batch(0, b)
            preds = getattr(self.loss_func, 'activation', noop)(self.pred)
            out = getattr(self.loss_func, 'decodes', noop)(preds)
            x,y,its,outs = self.dls.valid.show_results(b, out, show=False, max_n=self.n_preds)
            tensorboard_log(x, y, its, outs, self.writer, self.train_iter)

    def before_validate(self):
        if self.projector: self._setup_projector()

# Cell
class TensorBoardProjectorCallback(TensorBoardBaseCallback):
    "Extracts and exports image featuers for tensorboard projector during inference"
    def __init__(self, log_dir=None, layer=None):
        super().__init__()
        store_attr()

    def before_fit(self):
        self.run = not hasattr(self.learn, 'lr_finder') and hasattr(self, "gather_preds") and rank_distrib()==0
        if not self.run: return
        self._setup_writer()

    def before_validate(self):
        self._setup_projector()



# Cell
def _get_embeddings(model, layer):
    layer = model[0].encoder if layer == None else layer
    return layer.weight

# Cell
@typedispatch
def _normalize_for_projector(x:TensorImage):
    # normalize tensor to be between 0-1
    img = x.clone()
    sz = img.shape
    img = img.view(x.size(0), -1)
    img -= img.min(1, keepdim=True)[0]
    img /= img.max(1, keepdim=True)[0]
    img = img.view(*sz)
    return img




In [46]:
def _add_projector_features(learn, hook, feat):
    
    first_epoch = True if learn.iter == 0 else False
    
    #vec = hook.stored[0].max(1).values
    vec = torch.mean(hook.stored[0], dim=1)
    img = torch.full((len(vec),3,8,8), 0.7)
    
    #print('after pred')
    #print('x', learn.x)
    #print('has y', getattr(learn, 'y', None))
    #print('y', learn.y)
    dec = learn.dl.decode_batch((learn.x,learn.y), max_n=len(learn.x))
    #print('dec', len(dec), dec)
    dec_lists = list(map(list, zip(*dec)))
    lbls = dec_lists[1]
    texts = dec_lists[0]
    #print('lbls', lbls)
    lbl_texts = [[l,texts[i][:1000].replace('\t','').replace('\n','').replace('xxbos ','').replace('xxmaj ','')] for i, l in enumerate(lbls)]

    
    feat['vec'] = vec if first_epoch else torch.cat((feat['vec'], vec),0)
    feat['img'] = img           if first_epoch else torch.cat((feat['img'], img),0)
    #print(feat)
    if getattr(learn.dl, 'vocab', None):
        if 'lbl' not in feat:
            print('first epoch')
            feat['lbl'] = lbl_texts #if first_epoch else torch.cat((feat['lbl'], learn.y),0)
        else:
            print('other epochs')
            feat['lbl'] += lbl_texts
        
        
    #print('feat', feat)
    return feat

In [47]:
def _write_projector_embedding(learn, writer, feat):
    lbls = feat['lbl']
    #print('len lbls', len(lbls))
    #print('one lbl', lbls[0])
    #lbls = [learn.dl.vocab[l] for l in feat['lbl']] if getattr(learn.dl, 'vocab', None) else None
    #lbls = [[learn.dl.vocab[1][l],'bla blubb bla'] for l in feat['lbl']] if getattr(learn.dl, 'vocab', None) else None
    vecs = feat['vec'].squeeze()
    mh = ['label','text']
    print(len(lbls[0]), len(mh))
    writer.add_embedding(vecs, metadata=lbls, label_img=feat['img'], global_step=learn.train_iter, 
                         metadata_header=mh)

In [48]:
class TBSCB(TensorBoardBaseCallback):
    "Extracts and exports image featuers for tensorboard projector during inference"
    def __init__(self, log_dir=None, layer=None):
        super().__init__()
        store_attr()
        
    def after_pred(self):
        if self.run_projector: self.feat = _add_projector_features(self.learn, self.h, self.feat)

    def before_fit(self):
        self.run = not hasattr(self.learn, 'lr_finder') and hasattr(self, "gather_preds") and rank_distrib()==0
        if not self.run: return
        self._setup_writer()

    def before_validate(self):
        self._setup_projector()

In [50]:
dl = learn.dls.test_dl(df[:500], with_labels=True)

In [51]:
cbs = [TBSCB(layer=learn.model[0])]
_ = learn.get_preds(dl=dl, cbs=cbs)

first epoch
other epochs
other epochs
other epochs
other epochs
other epochs
other epochs
other epochs
2 2


In [189]:
b = dl.one_batch()

In [190]:
b

(TensorText([[   2,    8, 3544,  ...,  125,  186,   10],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         ...,
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1]]),
 TensorCategory([1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1]))

In [191]:
#o = dl.decode_batch(b)

In [192]:
fb = dl.before_batch.decode

In [193]:
fi = dl.after_item.decode

In [186]:
b[0]

TensorText([[   2,    8, 3544,  ...,  125,  186,   10],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])

In [199]:
o = fb(b[0])

In [200]:
o

TensorText([   2,    8, 3544,  ..., 5065,  233,   54])

In [201]:
o = fi(o)

In [202]:
o

TensorText([   2,    8, 3544,  ..., 5065,  233,   54])

In [203]:
o = fb(o)

In [204]:
o


TensorText([   2,    8, 3544,  ..., 5065,  233,   54])